In [1]:
print("all ok")

all ok


In [2]:
import weaviate
import os
from dotenv import load_dotenv
from weaviate.classes.init import Auth
load_dotenv()
WEAVIATE_KEY=os.getenv("WEAVIATE_API_KEY")
WEAVIATE_URL=os.getenv("WEAVIATE_URL")

HF_TOKEN=os.getenv("HF_API_TOKEN")

In [3]:
WEAVIATE_URL

'i89dhha7t7wg488bixluya.c0.asia-southeast1.gcp.weaviate.cloud'

In [4]:
headers = {
    "X-HuggingFace-Api-Key": HF_TOKEN,
}

In [5]:
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,                       # `weaviate_url`: your Weaviate URL
    auth_credentials=Auth.api_key(WEAVIATE_KEY),      # `weaviate_key`: your Weaviate API key
    headers=headers
)

In [6]:
client.is_ready()

True

In [12]:
client.collections.get('hybrid-search')

In [13]:
from weaviate.classes.config import Configure

In [14]:
client.collections.create(
    "HybridCollection",
    vector_config=[
        Configure.Vectors.text2vec_huggingface(
            name="title_vector",
            source_properties=["title"],
            # NOTE: Use only one of (`model`), (`passage_model` and `query_model`), or (`endpoint_url`)
            model="sentence-transformers/all-MiniLM-L6-v2",
            # passage_model="sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base",    # Required if using `query_model`
            # query_model="sentence-transformers/facebook-dpr-question_encoder-single-nq-base", # Required if using `passage_model`
            # endpoint_url="<custom_huggingface_url>",
            #
            # wait_for_model=True,
            # use_cache=True,
            # use_gpu=True,
        )
    ],
    # Additional parameters not shown
)

In [15]:
source_objects = [
    {"title": "The Shawshank Redemption", "description": "A wrongfully imprisoned man forms an inspiring friendship while finding hope and redemption in the darkest of places."},
    {"title": "The Godfather", "description": "A powerful mafia family struggles to balance loyalty, power, and betrayal in this iconic crime saga."},
    {"title": "The Dark Knight", "description": "Batman faces his greatest challenge as he battles the chaos unleashed by the Joker in Gotham City."},
    {"title": "Jingle All the Way", "description": "A desperate father goes to hilarious lengths to secure the season's hottest toy for his son on Christmas Eve."},
    {"title": "A Christmas Carol", "description": "A miserly old man is transformed after being visited by three ghosts on Christmas Eve in this timeless tale of redemption."}
]

In [16]:
collection = client.collections.use("HybridCollection")

In [17]:
with collection.batch.fixed_size(batch_size=200) as batch:
    for src_obj in source_objects:
        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties={
                "title": src_obj["title"],
                "description": src_obj["description"],
            },
            # vector=vector  # Optionally provide a pre-obtained vector
        )
        if batch.number_errors > 10:
            print("Batch import stopped due to excessive errors.")
            break

failed_objects = collection.batch.failed_objects
if failed_objects:
    print(f"Number of failed imports: {len(failed_objects)}")
    print(f"First failed object: {failed_objects[0]}")

#### Search

In [18]:
collection = client.collections.use("HybridCollection")

response = collection.query.near_text(
    query="A holiday film",  # The model provider integration will automatically vectorize the query
    limit=2
)
print(response)


QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('48cda31a-841c-4f36-b50f-99c45cf56f4f'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'title': 'A Christmas Carol', 'description': 'A miserly old man is transformed after being visited by three ghosts on Christmas Eve in this timeless tale of redemption.'}, references=None, vector={}, collection='HybridCollection'), Object(uuid=_WeaviateUUIDInt('f44f1ef5-3bdd-4093-81d5-bcce8dfa9ec4'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'title': 'The Shawshank Redemption', 'description': 'A wrongfully imprisoned man forms an inspiring friendship while finding hope and redemption in the darkest of places.'}, references=None, vector={}, collection='HybridCollection')])


In [19]:
for obj in response.objects:
    print(obj.properties["title"])

A Christmas Carol
The Shawshank Redemption


#### Hybrid Search

In [20]:
response = collection.query.hybrid(
    query="A holiday film",  # The model provider integration will automatically vectorize the query
    limit=2
)

for obj in response.objects:
    print(obj.properties["title"])

A Christmas Carol
The Shawshank Redemption


#### Working on pdf file

In [26]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader(r'F:\Tapas\Learning\GenAI\resource\research_paper\2401.15884v3.pdf')
data=loader.load()

In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

docs=splitter.split_documents(data)

In [28]:
len(docs)

144

In [29]:
docs[0]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-10-08T01:19:39+00:00', 'author': '', 'keywords': '', 'moddate': '2024-10-08T01:19:39+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'F:\\Tapas\\Learning\\GenAI\\resource\\research_paper\\2401.15884v3.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='Corrective Retrieval Augmented Generation\nShi-Qi Yan1*, Jia-Chen Gu2*, Yun Zhu3, Zhen-Hua Ling1\n1National Engineering Research Center of Speech and Language Information Processing,\nUniversity of Science and Technology of China, Hefei, China\n2Department of Computer Science, University of California, Los Angeles\n3Google DeepMind\nyansiki@mail.ustc.edu.cn, gujc@ucla.edu, yunzhu@google.com, zhling@ustc.edu.cn\nAbstract\nLarge language models (LLMs) inevitably')

In [33]:
client.collections.create(
    "HybridCollection",
    vector_config=[
        Configure.Vectors.text2vec_huggingface(
            name="content_vector",
            source_properties=["page_content"],
            # NOTE: Use only one of (`model`), (`passage_model` and `query_model`), or (`endpoint_url`)
            model="sentence-transformers/all-MiniLM-L6-v2",
            # passage_model="sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base",    # Required if using `query_model`
            # query_model="sentence-transformers/facebook-dpr-question_encoder-single-nq-base", # Required if using `passage_model`
            # endpoint_url="<custom_huggingface_url>",
            #
            # wait_for_model=True,
            # use_cache=True,
            # use_gpu=True,
        )
    ],
    # Additional parameters not shown
)

In [34]:
collection = client.collections.use("HybridCollection")

In [36]:
with collection.batch.fixed_size(batch_size=200) as batch:
    for src_obj in docs:
        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties={
                
                "conetnt": src_obj.page_content,
            },
            # vector=vector  # Optionally provide a pre-obtained vector
        )
        if batch.number_errors > 10:
            print("Batch import stopped due to excessive errors.")
            break

failed_objects = collection.batch.failed_objects
if failed_objects:
    print(f"Number of failed imports: {len(failed_objects)}")
    print(f"First failed object: {failed_objects[0]}")

In [42]:
response = collection.query.near_text(
    query="what is rag",  # The model provider integration will automatically vectorize the query
    limit=2
)
#print(response)

for obj in response.objects:
    print(obj.properties["conetnt"])


and Noah Fiedel. 2023. Palm: Scaling language
modeling with pathways. J. Mach. Learn. Res. ,
24:240:1–240:113.
Shehzaad Dhuliawala, Mojtaba Komeili, Jing Xu,
Roberta Raileanu, Xian Li, Asli Celikyilmaz, and
Jason Weston. 2024. Chain-of-verification reduces
hallucination in large language models. pages 3563–
3578.
Yann Dubois, Xuechen Li, Rohan Taori, Tianyi Zhang,
Ishaan Gulrajani, Jimmy Ba, Carlos Guestrin, Percy
Liang, and Tatsunori B. Hashimoto. 2023. Alpaca-
inference. The label of positive samples was 1,
while that of negative ones was -1. At inference,
the evaluator scored the relevance from -1 to 1 for
each document. The two confidence thresholds
for triggering one of the three actions were set
empirically. Specifically, they were set as (0.59,
-0.99) in PopQA, (0.5, -0.91) in PubQA and Arc-
Challenge, as well as (0.95, -0.91) in Biography.
Internal Knowledge: To obtain fine-grained
retrieval results, we segmented the retrieved results


In [45]:
response = collection.query.hybrid(
    query="What is RAG?",  # The model provider integration will automatically vectorize the query
    limit=2
)

for obj in response.objects:
    print(obj.properties["conetnt"],sep='\n\n')

play approach can be utilized in other concurrent
studies, we specifically designed to insert our
CRAG into the Self-RAG (Asai et al., 2024)
framework and named it Self-CRAG. Self-RAG
et al., 2020) pre-trained model. The dataset we
used is the version provided by Self-RAG (Asai
et al., 2024). Specifically, the original PopQA
dataset consists of 14k samples, 1,399 of which
were used for testing following Self-RAG (Asai
et al., 2024), and the remaining were used for
fine-tuning to avoid information leakage. Besides,
the fine-tuned evaluator was transferred and also
utilized on the Bio, Pub and ARC datasets during
inference. The label of positive samples was 1,
